In [1]:
import pandas as pd
import folium
import numpy as np
from sklearn.preprocessing import LabelEncoder 

In [2]:
foursquare = pd.read_csv('data/foursquare_complete.csv')
foursquare['geometry'] = list(zip(foursquare['lat'], foursquare['lon']))

enc = LabelEncoder()
enc.fit(foursquare['venue_category_name'])
foursquare['venue_category_name'] = enc.transform(foursquare['venue_category_name'])

#user file for recbole
pd.DataFrame(set(foursquare['uid']), columns=['uid:token']).to_csv('foursquare/foursquare.user', index=False, sep= '\t')


In [3]:
#item file for recbole
items = pd.DataFrame(set(foursquare['venue_id']), columns=['venue_id']).merge(foursquare[['venue_id', 'venue_category_name']], how = 'inner').drop_duplicates()
items.columns=['venue_id:token', 'venue_category_name:token']
items.to_csv('foursquare/foursquare.item', index = False, sep='\t')

In [5]:
import pickle

In [6]:
mapping = dict(zip(range(len(enc.classes_)), enc.classes_))
with open('saved_dictionary.pkl', 'wb') as f:
    pickle.dump(mapping, f)

In [7]:
#equal length in all of the trajectory
min_len = float('inf')
set_uid = set(foursquare['uid'])
for u in set_uid:
    min_len = min(min_len, len(foursquare[foursquare['uid'] == u]))

new_df = []
for u in set_uid:
    to_append = foursquare[foursquare['uid'] == u].iloc[:min_len, :].values.tolist()
    for r in to_append:
        new_df.append(r)

new_df = pd.DataFrame(new_df, columns=foursquare.columns)

#timestamp
new_df['timestamp'] = np.arange(0, 100).tolist() * len(set_uid)

#inter file for recbole
red_df = new_df[['uid', 'venue_id', 'timestamp']]
red_df.columns = ('uid:token', 'venue_id:token', 'timestamp:token')
red_df.to_csv('foursquare/foursquare.inter', index=False, sep = '\t')